# 2-3-ggplot2によるデータの可視化

In [ ]:
# ===============================================================
# ggplot2によるデータの可視化 (R) を Python に写経
# - Data I/O: pandas
# - Viz: matplotlib / seaborn / arviz
# - Labels: English
# - Bayesian estimation: NumPyro
# - Bayesian model visualization: NumPyro built-ins (render_model)
# - Posterior visualization: ArviZ (use hdi_prob)
# - Use print() when showing calculation results
# ===============================================================
# 必要なら以下を先にインストール
# pip install pandas numpy matplotlib seaborn arviz jax jaxlib numpyro graphviz statsmodels

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import arviz as az

import jax
import jax.numpy as jnp
import numpyro
import numpyro.distributions as dist
from numpyro.infer import NUTS, MCMC
# NumPyroのモデル可視化（Plate/Graph図）
try:
    from numpyro.contrib.render import render_model
    _HAS_RENDER = True
except Exception:
    _HAS_RENDER = False

# 表示系の初期化
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (7, 4)

# ---------------------------------------------------------------
# データの読み込み ------------------------------------------------
# fish: CSV (R: read.csv)
# ---------------------------------------------------------------
fish = pd.read_csv("2-2-1-fish.csv")  # <- 同名CSVを作業ディレクトリに置いてください
print("fish.head(3):")
print(fish.head(3))  # 計算(抽出)結果の表示は print()

# ---------------------------------------------------------------
# ヒストグラムとカーネル密度推定 -----------------------------------
# ---------------------------------------------------------------

# Histogram
plt.figure()
sns.histplot(fish["length"], bins=20, alpha=0.5)
plt.title("Histogram")
plt.xlabel("Length")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

# Kernel Density Estimate
plt.figure()
sns.kdeplot(fish["length"], linewidth=1.5)
plt.title("Kernel Density Estimate")
plt.xlabel("Length")
plt.ylabel("Density")
plt.tight_layout()
plt.show()

# Overlay: Histogram + KDE（密度スケールで重ね書き）
plt.figure()
sns.histplot(fish["length"], bins=20, alpha=0.5, stat="density")
sns.kdeplot(fish["length"], linewidth=1.5)
plt.title("Overlay: Histogram + KDE")
plt.xlabel("Length")
plt.ylabel("Density")
plt.tight_layout()
plt.show()

# グリッドで並べて表示（R: grid.arrange）
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
sns.histplot(fish["length"], bins=20, alpha=0.5, ax=axes[0])
axes[0].set_title("Histogram")
axes[0].set_xlabel("Length")
axes[0].set_ylabel("Count")

sns.kdeplot(fish["length"], linewidth=1.5, ax=axes[1])
axes[1].set_title("Kernel Density Estimate")
axes[1].set_xlabel("Length")
axes[1].set_ylabel("Density")
plt.tight_layout()
plt.show()

# ---------------------------------------------------------------
# 箱ひげ図とバイオリンプロット --------------------------------------
# R: iris データ
# ---------------------------------------------------------------
iris = sns.load_dataset("iris")  # 列名: sepal_length, petal_length, species など
print("iris.head(3):")
print(iris.head(3))

# Boxplot
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
sns.boxplot(data=iris, x="species", y="petal_length", ax=axes[0])
axes[0].set_title("Boxplot")
axes[0].set_xlabel("Species")
axes[0].set_ylabel("Petal Length")

# Violin plot
sns.violinplot(data=iris, x="species", y="petal_length", ax=axes[1])
axes[1].set_title("Violin Plot")
axes[1].set_xlabel("Species")
axes[1].set_ylabel("Petal Length")

plt.tight_layout()
plt.show()

# ---------------------------------------------------------------
# 散布図 ----------------------------------------------------------
# ---------------------------------------------------------------
# Simple scatter
plt.figure()
sns.scatterplot(data=iris, x="petal_width", y="petal_length")
plt.title("Scatter Plot")
plt.xlabel("Petal Width")
plt.ylabel("Petal Length")
plt.tight_layout()
plt.show()

# Colored scatter by species
plt.figure()
sns.scatterplot(data=iris, x="petal_width", y="petal_length", hue="species")
plt.title("Scatter Plot by Species")
plt.xlabel("Petal Width")
plt.ylabel("Petal Length")
plt.tight_layout()
plt.show()

# ---------------------------------------------------------------
# 折れ線グラフ（ナイル川流量） --------------------------------------
# R: Nile, data.frame(year=1871:1970, Nile=as.numeric(Nile))
# ---------------------------------------------------------------
import statsmodels.api as sm
nile_df = sm.datasets.nile.load_pandas().data  # 列: 'year', 'volume'
print("nile_df.head(3):")
print(nile_df.head(3))

# Line plot
plt.figure()
plt.plot(nile_df["year"], nile_df["volume"])
plt.title("Line Plot: Nile River Flow")
plt.xlabel("Year")
plt.ylabel("Flow")
plt.tight_layout()
plt.show()

# ts オブジェクト風の簡易描画（年をインデックスに）
nile_series = nile_df.set_index("year")["volume"]
plt.figure()
nile_series.plot()
plt.title("Time Series: Nile River Flow")
plt.xlabel("Year")
plt.ylabel("Flow")
plt.tight_layout()
plt.show()

# ---------------------------------------------------------------
# ベイズ推定 (NumPyro) ---------------------------------------------
# 例: 魚の length を正規分布でモデル化
#   y_i ~ Normal(mu, sigma)
#   mu ~ Normal(0, 10)
#   sigma ~ HalfNormal(5)
# 事後分布の可視化は ArviZ で az.plot_posterior(..., hdi_prob=...)
# モデル可視化は NumPyro の render_model を使用
# ---------------------------------------------------------------

# 観測データ（NaN除去、JAX配列へ）
y = jnp.asarray(fish["length"].dropna().values)

def model(y_obs):
    mu = numpyro.sample("mu", dist.Normal(0.0, 10.0))
    sigma = numpyro.sample("sigma", dist.HalfNormal(5.0))
    numpyro.sample("obs", dist.Normal(mu, sigma), obs=y_obs)

# モデルの可視化（Graphviz 必要）
if _HAS_RENDER:
    try:
        dot = render_model(model, model_args=(y,), render_distributions=True, render_params=True)
        # ファイルに保存（PNG）
        dot.render("numpyro_model_graph", format="png", cleanup=True)
        # 画像として表示
        if os.path.exists("numpyro_model_graph.png"):
            img = plt.imread("numpyro_model_graph.png")
            plt.figure(figsize=(6, 6))
            plt.imshow(img)
            plt.axis("off")
            plt.title("Model Graph (NumPyro)")
            plt.tight_layout()
            plt.show()
    except Exception as e:
        print(f"[Info] Model rendering skipped: {e}")

# 乱数キー（再現性）
rng_key = jax.random.PRNGKey(0)

# NUTS + MCMC
nuts = NUTS(model)
mcmc = MCMC(nuts, num_warmup=1000, num_samples=2000, num_chains=2, progress_bar=True)
mcmc.run(rng_key, y)

# サマリー（print()で表示）
print("MCMC summary:")
mcmc.print_summary()

# ArviZ へ変換して統計量も print（HDI=0.95）
idata = az.from_numpyro(mcmc)
summary_df = az.summary(idata, var_names=["mu", "sigma"], hdi_prob=0.95)
print("ArviZ summary (95% HDI):")
print(summary_df)

# 事後分布の可視化（credible_interval は使わず、hdi_prob を使用）
plt.figure(figsize=(8, 4))
az.plot_posterior(idata, var_names=["mu", "sigma"], hdi_prob=0.95)
plt.suptitle("Posterior Distributions (95% HDI)", y=1.02)
plt.tight_layout()
plt.show()

# 追加でトレース（任意）
plt.figure(figsize=(8, 6))
az.plot_trace(idata, var_names=["mu", "sigma"])
plt.suptitle("Trace Plots", y=1.02)
plt.tight_layout()
plt.show()

# ---------------------------------------------------------------
# ggplot2 まとめの疑似コード相当（Python版の書式メモ）
# ---------------------------------------------------------------
# # seaborn の基本形（擬似コード）
# sns.XXXplot(data=<DataFrame>, x="X", y="Y", hue="Group", ...)
# plt.title("Title")
# plt.xlabel("X label")
# plt.ylabel("Y label")
# plt.show()
